In [2]:
import Pkg; 

cd(joinpath(@__DIR__, "../../"))
Pkg.activate("Project.toml")

using MorphoMol
using PyCall
using JLD2
using LinearAlgebra
using NearestNeighbors
using Rotations

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [17]:
function get_interface_diagram(points, n_atoms_per_mol)
    py"""
    import oineus as oin
    import numpy as np
    import torch
    import diode

    def get_interface_diagram(points, n_atoms_per_mol):
        points = np.asarray(points)
        simplices = diode.fill_alpha_shapes(points)
        fil = oin.Filtration_double([oin.Simplex_double(s[0], s[1]) for s in simplices])

        def is_multi(sigma):
            has_a = has_b = False
            for v in sigma.vertices:
                if v < n_atoms_per_mol:
                    has_a = True
                else:
                    has_b = True
            return has_a and has_b

        fil = fil.subfiltration(is_multi)
        # second argument: True for cohomology, False for homology (incorrect for subfiltration)
        dcmp = oin.Decomposition(fil, True)
        params = oin.ReductionParams()
        params.clearing_opt = False
        dcmp.reduce(params)
        dgm = dcmp.diagram(fil, include_inf_points=False)
        return dgm
    """
    py"get_interface_diagram"(points, n_atoms_per_mol)
end

function get_interface_diagram_as_numpy_false(points, n_atoms_per_mol)
    py"""
    import oineus as oin
    import numpy as np
    import torch
    import diode

    def get_interface_diagram(points, n_atoms_per_mol):
        points = np.asarray(points)
        simplices = diode.fill_alpha_shapes(points)
        fil = oin.Filtration_double([oin.Simplex_double(s[0], s[1]) for s in simplices])

        def is_multi(sigma):
            has_a = has_b = False
            for v in sigma.vertices:
                if v < n_atoms_per_mol:
                    has_a = True
                else:
                    has_b = True
            return has_a and has_b

        fil = fil.subfiltration(is_multi)
        # second argument: True for cohomology, False for homology (incorrect for subfiltration)
        dcmp = oin.Decomposition(fil, True)
        params = oin.ReductionParams()
        params.clearing_opt = False
        dcmp.reduce(params)
        dgm = dcmp.diagram(fil, as_numpy = False, include_inf_points=False)
        return dgm
    """
    py"get_interface_diagram"(points, n_atoms_per_mol)
end

get_interface_diagram_as_numpy_false (generic function with 1 method)

In [5]:
@load "../../Data/measure_paths/jld2s/6r7m/19.jld2" dgms tps Es thetas realizations
#tps = [get_total_persistence(dim_dgms, -0.1) for dim_dgms in dgms] #Recalculating due to weights!
xs = [i for i in 1:length(dgms)];
two_ps = [MorphoMol.Energies.get_persistence(dgm[2], -0.1) for dgm in dgms]
three_ps = [MorphoMol.Energies.get_persistence(dgm[3], -0.1) for dgm in dgms]
tps = [MorphoMol.Energies.get_total_persistence(dgm, -0.1) for dgm in dgms];

In [19]:
points = mapreduce(permutedims, vcat, [realizations[1][1]; realizations[1][2]])

2412×3 Matrix{Float64}:
 192.146  17.5403  108.954
 192.159  18.7326  109.789
 190.793  19.4012  109.816
 189.944  19.1296  108.971
 193.209  19.7276  109.296
 192.832  20.2909  108.052
 190.583  20.266   110.8
 189.439  21.1632  110.81
 189.815  22.463   110.111
 190.96   22.9147  110.183
 188.986  21.4729  112.237
 188.275  20.3565  112.962
 187.042  19.8928  112.526
   ⋮               
 142.614  27.2821   77.3803
 142.155  28.1685   79.5785
 143.658  28.174    77.1878
 143.19   29.0494   79.3854
 143.927  29.0492   78.1984
 138.316  27.7224   77.2578
 137.77   27.815    75.9117
 138.885  28.0611   74.9054
 139.608  29.0511   75.0005
 136.726  28.9354   75.8019
 135.58   28.6005   76.5924
 136.294  29.1243   74.3553

In [21]:
get_interface_diagram(points, 1206)

PyObject <oineus._oineus.Diagrams_double object at 0x2bc72d4b0>